In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

import sys, os
from os.path import join
sys.path.append('/work/mflora/ROAD_SURFACE')
sys.path.insert(0, '/home/monte.flora/python_packages/scikit-explain')


from skexplain.plot.base_plotting import PlotStructure
from matplotlib.lines import Line2D
from skexplain import ExplainToolkit
from skexplain.common.importance_utils import to_skexplain_importance

from ml_workflow.ml_methods import norm_aupdc, brier_skill_score
from ml_workflow import CalibratedPipelineHyperOptCV
from ml_workflow.ml_methods import get_bootstrap_score
from calibration_classifier import CalibratedClassifier
from sklearn.compose import TransformedTargetRegressor

from wofs_ml_severe.wofs_ml_severe.common.multiprocessing_utils import run_parallel, to_iterator

from sklearn.metrics import r2_score, mean_squared_error
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from os.path import join
import pickle
from joblib import load
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import RidgeCV


from sklearn.ensemble import RandomForestClassifier
import hyperopt.hp as hp
import seaborn as sns
import itertools
sns.set_theme(palette=sns.color_palette("Set2"))

from scipy.stats import spearmanr, kendalltau
from probsr_config import TARGET_COLUMN, PREDICTOR_COLUMNS

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
#model_name = f'../models/RandomForest_manualfeatures_12.joblib'
#model = load(model_name)['model'] 

In [3]:
#model

In [4]:
BASE_PATH = '/work/mflora/explainability_work/'
DATA_BASE_PATH = join(BASE_PATH, 'datasets')
MODEL_BASE_PATH = join(BASE_PATH, 'models')

## Create the Association between feature importance and model performance. 

In [5]:
# Load in the respective ranking method results
other_base_path = '/work/mflora/explainability_work/results'
def get_feature_importances(target, opt, method, results=None):
    
    time='first_hour'
    method = 'unnormalized'
    explainer = ExplainToolkit()
    if target == 'road_surface':
        ###opt = 'reduced'
        est_name = 'Random Forest'
        if opt == 'reduced':
            model_name = join(MODEL_BASE_PATH,'RandomForest_manualfeatures_12.joblib')
            model = load(model_name)['model'] 
            data = load(model_name)
            model = data['model'] 
            features = data['features']
            gini_values = model.base_estimator.named_steps['model'].feature_importances_
        else:
            model_name = join(MODEL_BASE_PATH,'JTTI_ProbSR_RandomForest.pkl')
            model = load(model_name)
            gini_values = model.feature_importances_
            features = PREDICTOR_COLUMNS
        
        pimp_backward_fname = join(DATA_BASE_PATH, f'perm_imp_{opt}_backward.nc')
        pimp_forward_fname = join(DATA_BASE_PATH, f'perm_imp_{opt}_forward.nc')
        ale_var_fname = join(DATA_BASE_PATH, f'ale_var_rf_{opt}.nc')
        shap_fname = join(DATA_BASE_PATH, f'shap_rf_{opt}.nc')

        gini_rank = to_skexplain_importance(gini_values,
                                       estimator_name='Random Forest', 
                                       feature_names=features, 
                                         method = 'gini')
        
    else:
        est_name = 'LogisticRegression'
        model_name = join(MODEL_BASE_PATH, 
                          f'LogisticRegression_first_hour_{target}_under_standard_{opt}.pkl')
        data = load(model_name)
        model = data['model']
        features = data['features']
    
        pimp_backward_fname = join(DATA_BASE_PATH,
                        f'permutation_importance_{target}_{time}_training_norm_aupdc{opt}backward.nc')
        pimp_forward_fname = join(DATA_BASE_PATH, 
                        f'permutation_importance_{target}_{time}_training_norm_aupdc{opt}forward.nc')
        ale_var_fname = join(DATA_BASE_PATH,
                        f'ale_var_results_all_models_{target}_{time}{opt}.nc')
        shap_fname = join(DATA_BASE_PATH,
                        f'shap_values_LogisticRegression_{target}_{time}{opt}.pkl')

        coefs = model.base_estimator.named_steps['model'].coef_[0]
        coef_rank = to_skexplain_importance(coefs,
                                       estimator_name='LogisticRegression', 
                                       feature_names=features, 
                                       method = 'coefs')
        
        
        
        
    
    
    sage_opt = 'reduced' if ('L1_based' in opt or opt=='reduced') else 'original'
    sage_fname = join(DATA_BASE_PATH,f'sage_results_{sage_opt}_{target}.nc')
    ale_var = explainer.load(ale_var_fname)
    pimp_backward = explainer.load(pimp_backward_fname)
    pimp_forward = explainer.load(pimp_forward_fname)
    
    with open(shap_fname, 'rb') as pkl_file:
        data = pickle.load(pkl_file)

    X = data['X']
    shap_vals = data['shap_values']
    shap_rank = to_skexplain_importance(shap_vals, 
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method ='shap_sum')

    with open(sage_fname, 'rb') as f:
        sage_results = pickle.load(f)
    
    sage_rank = to_skexplain_importance(sage_results,
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method = 'sage')
    
    # Lime
    lime_fname = join(other_base_path,f'lime_rank_{target}_{sage_opt}.nc')
    lime_rank = explainer.load(lime_fname)
    
    # TI 
    if target=='road_surface':
        ti_fname = join(other_base_path,f'ti_rank_{target}_{sage_opt}.nc')
        ti_rank = explainer.load(ti_fname)
    
    
    data = [pimp_backward, 
            pimp_backward,
            pimp_forward, 
            pimp_forward,
            ale_var,  
            shap_rank,
            sage_rank,
            lime_rank
           ]
    
    methods = ['multipass__backward', 
           'singlepass__backward', 
           'multipass__forward', 
           'singlepass__forward', 
           'ale_variance', 
           'shap_sum',
           'sage',
           'lime'
           ]

    if target == 'road_surface':
        data.append(ti_rank)
        methods.append('tree_interpreter')
    
    if target == 'road_surface': 
        data.append(gini_rank)
        methods.append('gini')
    else:
        data.append(coef_rank)
        methods.append('coefs')
    
    # Turn the permutation importances into proper importance scores. 
    # Backward multipass and forward singlepass: 1 + (permute score - original score)
    original_score = pimp_backward[f'original_score__{est_name}'].values
    scores = 1.0 + (pimp_backward[f'multipass_scores__{est_name}'].values - original_score)
    pimp_backward[f'multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], scores)
    
    original_score = pimp_forward[f'original_score__{est_name}'].values
    scores = 1.0 + (pimp_forward[f'singlepass_scores__{est_name}'].values - original_score)
    pimp_forward[f'singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], scores)
  
    print(pimp_backward[f'multipass_scores__{est_name}'].shape)
    print(pimp_forward[f'multipass_scores__{est_name}'].shape)
    
    # Backward singlepass and forward multipass: original_score - permuted score
    original_score = pimp_backward[f'original_score__{est_name}'].values
    scores = original_score - pimp_backward[f'singlepass_scores__{est_name}'].values
    pimp_backward[f'singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], scores)
    
    original_score = pimp_forward[f'original_score__{est_name}'].values
    scores = original_score - pimp_forward[f'multipass_scores__{est_name}'].values
    pimp_forward[f'multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], scores)
    
    #data = [shap_rank, pimp_backward]
    #methods = ['shap_sum', 'multipass__backward']
    
    feature_importances = {m : pd.DataFrame(ds[f"{m.split('__')[0]}_scores__{est_name}"].values.T, 
                       columns = ds[f"{m.split('__')[0]}_rankings__{est_name}"]) 
                           for m,ds in zip(methods, data) }
    
    #for key in feature_importances.keys():
    #    print(key)
    #    print(f'{feature_importances[key]=}')
    
    if results is not None:
        feature_subsets = results['subsets']
        n_iter = len(feature_subsets)
        importances = {m : np.zeros((n_iter)) for m in methods}
         # Iterate over several thousand feature subsets
        for i, inds in enumerate(feature_subsets):
            for m in methods:
                # Sum up the importances from the different bootstrapping and then average. 
                # and then normalize by the feature set size. 
                if method == 'normalized':
                    factor =  float((len(features) - len(inds)) / len(features))
                elif method == 'normalized_divide':
                    factor = 1.0 / len(inds)
                else:
                    factor = 1.0
                
                val = np.sum(np.mean(feature_importances[m][inds], axis=0)) * factor
                
                importances[m][i] = val
    else:
        importances = None
    
    return feature_importances, importances 

### Train models in the top and bottom 15 predictors. 

In [17]:
def train_top_and_bottom_predictors(target, opt):
    N_FEATURES = 15
    
    model_opt = 'L1_based_feature_selection_with_manual' if opt == 'reduced' else ''
    
    data = {}
    time = 'first_hour'
    param_grid = {
                'l1_ratio': hp.choice('l1_ratio', [0.0001, 0.001, 0.01, 0.1, 0.5, 0.6, 0.8, 1.0]),
                'C': hp.choice('C', [0.0001, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]),
                }

    save_fname = join(DATA_BASE_PATH, f'top_worst_predictors_{target}_{model_opt}_scores.pkl')


    print(f'Working on {target}...')
    # Load training dataset
    #df = pd.read_pickle(f'../datasets/{time}_training_matched_to_{target}_0km_dataset')
    #dates = df['Run Date']
    #fti = df['FCST_TIME_IDX'].astype(int) 
    
    model_fname = join(MODEL_BASE_PATH,
                           f'LogisticRegression_first_hour_{target}_under_standard_{model_opt}.pkl')
    
    data = load(model_fname)
    original_features = data['features']
    model = data['model']
    params = model.base_estimator.named_steps['model'].get_params()
    params['max_iter'] = 300
    
    #X = df[original_features].astype(float)
    #y = df[f'matched_to_{target}_0km'].astype(float).values 
    #known_skew = np.mean(y)
    known_skew = 0.05
    dates = []
        
    fname = join(DATA_BASE_PATH, f'importances_{target}_{model_opt}.pkl')
    with open(fname, 'rb') as f:
        data = pickle.load(f)     
    
    feature_importances = data['feature_importances']
        
    feature_importances, _ = get_feature_importances(target, opt, results=None,)
    results = {m:{} for m in feature_importances.keys() }
    
    for m in feature_importances.keys():
        print('*'*40)
        print(f'\nMethod : {m}\n')
        print('*'*40)
        imp = feature_importances[m]
        
        best_5 = list(feature_importances[m].columns[:N_FEATURES])
        worst_5 = list(feature_importances[m].columns[-N_FEATURES:])
        
        estimator = LogisticRegression(solver='saga', 
                                       penalty='elasticnet', max_iter=300, random_state=42)

        top_scores = []
        for i, _ in enumerate(best_5):
            clf = CalibratedPipelineHyperOptCV( base_estimator = estimator,
                                  param_grid = param_grid,
                                  resample='under',
                                  scaler='standard',
                                  imputer=None,
                                  hyperopt='atpe',
                                  n_jobs=1,
                                  max_iter=20,
                                  scorer_kwargs = {'known_skew': known_skew },
                                  cv = 'date_based',
                                  cv_kwargs = {'n_splits' : 5,
                                               'dates' : dates,
                                               'valid_size' : 20 },
                                  )
            
            
            feature_subset = best_5[:i+1]
            print('\n', feature_subset)
            X_subset = X[feature_subset]
            clf.fit(X_subset, y, params=params)
            preds = clf.predict_proba(X_subset)[:,1]
            ds = get_bootstrap_score(y, preds, n_bootstrap=100, 
                                     known_skew = known_skew, 
                                     forecast_time_indices = fti,
                                     metric_mapper={'naupdc' : norm_aupdc})
            top_scores.append(ds)
            
        bottom_scores = []
        for i, _ in enumerate(worst_5[::-1]):
            clf = CalibratedPipelineHyperOptCV( base_estimator = estimator,
                                  param_grid = param_grid,
                                  resample='under',
                                  scaler='standard',
                                  imputer=None,
                                  hyperopt='atpe',
                                  n_jobs=1,
                                  max_iter=20,
                                  scorer_kwargs = {'known_skew': known_skew },
                                  cv = 'date_based',
                                  cv_kwargs = {'n_splits' : 5,
                                               'dates' : dates,
                                               'valid_size' : 20 },
                                  )
            feature_subset = worst_5[::-1][:i+1]
            X_subset = X[feature_subset]
            clf.fit(X_subset, y, params=params)
            preds = clf.predict_proba(X_subset)[:,1]
            ds = get_bootstrap_score(y, preds, n_bootstrap=100, 
                                     known_skew = known_skew, 
                                     forecast_time_indices = fti ,
                                     metric_mapper={'naupdc' : norm_aupdc})
            
            bottom_scores.append(ds)
            
        results[m]['top_scores'] = top_scores
        results[m]['bottom_scores'] = bottom_scores
        
    with open(save_fname, 'wb') as f:
        pickle.dump(results, f)

In [18]:
train_top_and_bottom_predictors('tornado', 'reduced')

Working on tornado...


FileNotFoundError: [Errno 2] No such file or directory: '/work/mflora/explainability_work/datasets/importances_tornado_L1_based_feature_selection_with_manual.pkl'

In [ ]:
#train_top_and_bottom_predictors()
#targets= ['tornado', 'severe_hail', 'severe_wind', 'road_surface']
#opts = ['L1_based_feature_selection_with_manual', '']

run_parallel(train_top_and_bottom_predictors, 
#            args_iterator=to_iterator(targets, opts), 
#             nprocs_to_use=len(targets)*len(opts), 
#             description='Feature Importance Experiment')

In [ ]:
def train_on_top_predictors(target, opt):
    param_grid = {
                'l1_ratio': hp.choice('l1_ratio', [0.0001, 0.001, 0.01, 0.1, 0.5, 0.6, 0.8, 1.0]),
                'C': hp.choice('C', [0.0001, 0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1.0]),
                }

        
    if target == 'road_surface':
        fti = None
        opt = 'original' if opt == '' else 'reduced'
        print(f'Working on {target} {opt}...')
        df = pd.read_csv('../datasets/probsr_training_data.csv')
        random_state = np.random.RandomState(123)
        inds = random_state.choice(len(df), size=300000, replace=False)
        dates = pd.to_datetime(df['date'], errors ='coerce')

        dates.astype('int64').dtypes
        # extracting the week from the date
        weekNumber = dates.dt.isocalendar().week
        weekNumber = weekNumber.values[inds]
        
        
        if opt == 'reduced':
            model_name = f'../models/RandomForest_manualfeatures_12.joblib'
            model = load(model_name)['model'] 
            data = load(model_name)
            model = data['model'] 
            features = data['features']
                
            X = df[features].astype(float)
            X = X.iloc[inds, :]
            X.reset_index(drop=True, inplace=True)
                
            y = df[TARGET_COLUMN].astype(float)
            y = y.values[inds]
            params = model.base_estimator.named_steps['model'].get_params()
                
            else:
                model_fname = '../models/JTTI_ProbSR_RandomForest.pkl'
                calibrator_fname = '../models/JTTI_ProbSR_RandomForest_Isotonic.pkl'
        
                base_estimator = load(model_fname)
                calibrator = load(calibrator_fname)
                model = CalibratedClassifier(base_estimator, calibrator)
                params = base_estimator.get_params()
        
                X = df[PREDICTOR_COLUMNS]
                X = X.iloc[inds, :]
                X.reset_index(drop=True, inplace=True)
                y = df[TARGET_COLUMN]
                y = y.values[inds]

    else:
        print(f'Working on {target} {opt}...')

        # Load training dataset
        df = pd.read_pickle(f'../datasets/{time}_training_matched_to_{target}_0km_dataset')
        dates = df['Run Date']
        fti = df['FCST_TIME_IDX'].astype(int) 
    
        model_fname = f'../models/LogisticRegression_first_hour_{target}_under_standard_{opt}.pkl'
    
        data = load(model_fname)
        original_features = data['features']
        model = data['model']
        params = model.base_estimator.named_steps['model'].get_params()
        params['max_iter'] = 300
    
        X = df[original_features].astype(float)
        y = df[f'matched_to_{target}_0km'].astype(float).values
    
    known_skew = np.mean(y)

    feature_importances, _ = get_feature_importances(target, opt, results=None,)
    results = {m:{} for m in feature_importances.keys() }
    
    n_features=10
    for key in ['top_scores', 'bottom_scores']:
        for m in feature_importances.keys():
            print(f'Method : {m}')
            imp = feature_importances[m]
            
            if key == 'top_scores':
                best_features = list(imp.columns[:n_features])
            else:
                best_features = list(imp.columns[-n_features:])
                
            if target == 'road_surface':
                estimator = RandomForestClassifier(n_jobs=10, 
                                                   random_state=30, 
                                                   criterion = 'entropy', 
                                                   class_weight = 'balanced')
                    cv_kwargs = {'n_splits' : 5,
                            'dates' : weekNumber,
                             'valid_size' : 8 }
                else:
                    estimator = LogisticRegression(solver='saga', 
                                       penalty='elasticnet', max_iter=300, random_state=42)
                    cv_kwargs = {'n_splits' : 5,
                             'dates' : dates,
                             'valid_size' : 20 }

                scores = []
                clf = CalibratedPipelineHyperOptCV( base_estimator = estimator,
                                  param_grid = param_grid,
                                  resample='under',
                                  scaler='standard',
                                  imputer=None,
                                  hyperopt='atpe',
                                  n_jobs=1,
                                  max_iter=20,
                                  scorer_kwargs = {'known_skew': known_skew },
                                  cv = 'date_based',
                                  cv_kwargs = cv_kwargs,
                                  )
            
                X_subset = X[best_features]
                clf.fit(X_subset, y, params=params)
                preds = clf.predict_proba(X_subset)[:,1]
                ds = get_bootstrap_score(y, preds, n_bootstrap=100, 
                                     known_skew = known_skew, 
                                     forecast_time_indices = fti,
                                     metric_mapper={'naupdc' : norm_aupdc, 
                                                    'bss' : brier_skill_score,
                                                   })
                scores.append(ds)
                results[m][key] = scores
            
        save_fname = f'../datasets/top_predictors_{target}_{opt}_scores.pkl'
        print(f'Saving {save_fname}...')
        with open(save_fname, 'wb') as f:
            pickle.dump(results, f)

In [7]:
import traceback

targets= ['tornado', 'severe_hail', 'severe_wind', 'road_surface']
opts = ['L1_based_feature_selection_with_manual', '']
names = [ 'unnormalized']

def func(target, opt, name):
    if target == 'road_surface':
        opt = 'original' if opt == '' else 'reduced'
        fname = join(DATA_BASE_PATH,f'RF_road_surface_{opt}__scores.pkl')
    else:
        fname = join(DATA_BASE_PATH,f'LogisticRegression_first_hour_{target}_{opt}__scores.pkl')
    
    try:
        print('fname: ', fname)
        with open(fname, 'rb') as f:
            results = pickle.load(f)
        
        feature_importances, importances = get_feature_importances(target, opt, 
                                                                   results=results,
                                                                   method=name
                                                                   )
    except:
        print(traceback.format_exc())
        results=[]
        importances=[]
        
    try:
        scores = np.array(results['scores'])
    except:
        scores=[]
        
    data = {'scores' : scores, 
            'feature_importances': feature_importances, 
            'importances' : importances
    }
    
    save_fname = join(DATA_BASE_PATH,f'importances_{target}_{opt}_{name}.pkl')
    
    print(f'Writing {save_fname}...')
    with open(save_fname, 'wb') as f:
        pickle.dump(data, f) 

In [8]:
#func('road_surface', '', 'unnormalized')

In [9]:
run_parallel(func, 
             args_iterator=to_iterator(targets, opts, names), 
             nprocs_to_use=len(targets)*len(opts)*len(names), 
             description='Feature Importance Experiment')

Feature Importance Experiment:   0%|                                                                                  | 0/8 [00:00<?, ?it/s]

fname: fname: fname: fname: fname: fname: fname: fname:        /work/mflora/explainability_work/datasets/LogisticRegression_first_hour_severe_hail_L1_based_feature_selection_with_manual__scores.pkl /work/mflora/explainability_work/datasets/LogisticRegression_first_hour_tornado_L1_based_feature_selection_with_manual__scores.pkl/work/mflora/explainability_work/datasets/LogisticRegression_first_hour_severe_hail___scores.pkl/work/mflora/explainability_work/datasets/LogisticRegression_first_hour_severe_wind_L1_based_feature_selection_with_manual__scores.pkl/work/mflora/explainability_work/datasets/LogisticRegression_first_hour_tornado___scores.pkl/work/mflora/explainability_work/datasets/LogisticRegression_first_hour_severe_wind___scores.pkl/work/mflora/explainability_work/datasets/RF_road_surface_original__scores.pkl/work/mflora/explainability_work/datasets/RF_road_surface_reduced__scores.pkl







(14, 1000)
(14, 1000)
(25, 1000)
(25, 1000)
(32, 1000)
(32, 1000)
(30, 500)
(30, 500)
(113,

Feature Importance Experiment:  12%|█████████▎                                                                | 1/8 [00:29<03:28, 29.75s/it]

Writing /work/mflora/explainability_work/datasets/importances_tornado_L1_based_feature_selection_with_manual_unnormalized.pkl...


Feature Importance Experiment:  25%|██████████████████▌                                                       | 2/8 [00:29<01:14, 12.35s/it]

...

Writing /work/mflora/explainability_work/datasets/importances_severe_wind__unnormalized.pkl...


Feature Importance Experiment:  88%|████████████████████████████████████████████████████████████████▊         | 7/8 [00:36<00:02,  2.06s/it]

Writing /work/mflora/explainability_work/datasets/importances_road_surface_reduced_unnormalized.pkl...


Feature Importance Experiment: 100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:54<00:00,  6.84s/it]


[None, None, None, None, None, None, None, None]

In [51]:
target = 'tornado'
opt = ''
name = 'unnormalized'

save_fname = join(DATA_BASE_PATH,f'importances_{target}_{opt}_{name}.pkl')
with open(save_fname, 'rb') as f:
    data = pickle.load(f) 

In [52]:
data['importances']

{'multipass__backward': array([84.27301339, 52.01720773, 71.43467199, ..., 84.25975586,
        19.9128116 , 67.39092928]),
 'singlepass__backward': array([0.22890405, 0.13480542, 0.17808643, ..., 0.2240216 , 0.05934309,
        0.17213575]),
 'multipass__forward': array([3.23199299, 2.33906528, 2.9362185 , ..., 3.28155818, 1.0577395 ,
        2.5376998 ]),
 'singlepass__forward': array([82.30490197, 50.42458624, 69.70266776, ..., 82.30457689,
        19.27929793, 65.98978412]),
 'ale_variance': array([0.23147406, 0.14182992, 0.20880368, ..., 0.23184216, 0.05566745,
        0.16313019]),
 'shap_sum': array([32.06791779, 21.56891473, 28.36262951, ..., 32.25782485,
         8.42136643, 24.85530154]),
 'sage': array([6.42116173, 4.77702071, 6.30638108, ..., 6.45591226, 1.70388128,
        4.7656822 ]),
 'coefs': array([33.73604638, 22.32935848, 29.36938716, ..., 34.12734362,
         9.2388022 , 26.61489306])}